# Load Data

In [ ]:
def make_table(df: pd.DataFrame, label = "") -> pd.DataFrame:
  progressBar = IntProgress(min=0, max=min(df['tupleid'].max(), MAXIMUM_ROW_COUNT), description=label)
  display(progressBar)

  grouped = df.groupby("tupleid")

  attributes = df['attribute'].unique()
  result = pd.DataFrame(np.nan, index=df['tupleid'].unique()[:MAXIMUM_ROW_COUNT], columns=attributes)
  for name, group in grouped:
    if name > MAXIMUM_ROW_COUNT:
      break
    progressBar.value += 1
    result.loc[name] = [group.loc[group["attribute"] == attribute, "value"].item() for attribute in attributes]
    # try:
    #   result.loc[name] = [group.loc[group["attribute"] == attribute, "value"].item() for attribute in attributes]
    # except Exception as e:
    #   print([group.loc[group["attribute"] == attribute, "value"] for attribute in attributes])
    #   break
  return result

#def make_error_table(dirty: pd.DataFrame, clean: pd.DataFrame) -> pd.DataFrame:
#return dirty.where(dirty == clean, True)

# Old Experiments

In [ ]:
import pandas as pd

hospital_dirty = pd.read_csv("hospital_input.csv")
hospital_clean = pd.read_csv("hospital_clean.csv")

In [ ]:
def serialize_row(row: pd.Series) -> str:
  result = ""
  for index, value in row.items():
    result += f"{index}: {value} "
  return result

def serialize_table(table: pd.DataFrame, max_rows = 10) -> str:
  result = ""
  counter = 0
  for index, row in table.iterrows():
    counter += 1
    if counter > max_rows:
      return result
    result += serialize_row(row)
    result += "\n"

def serialize_hospital_table(table: pd.DataFrame, max_rows) -> str:
  result = ""
  counter = 0
  current_tuple = 0
  for index, row in table.iterrows():
    if current_tuple != row["tupleid"]:
      counter += 1
      if counter > max_rows:
        return result
      current_tuple = row["tupleid"]
      result += f"\nROW: {current_tuple} "
    result += f"{row['attribute']}: "
    result += f"{row['value']} "

In [ ]:
print(ask_llama(f"""Take these data entries as the ground truth:

{serialize_hospital_table(hospital_clean, 5)}

Find all the errors in the following entry:

{serialize_hospital_table(hospital_dirty, 1)}.

Give me the tag of the value aswell."""))

SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

  USER: Take these data entries as the ground truth:


ROW: 1 provider number: 10018 hospital name: CALLAHAN EYE FOUNDATION HOSPITAL address1: 1720 UNIVERSITY BLVD address2: nan address3: nan city: BIRMINGHAM state: AL zip code: 35233 county name: JEFFERSON phone number: 2053258100 hospital type: Acute Care Hospitals hospital owner: Voluntary non-profit - Private emergency service: Yes condition: Surgical Infection Prevention measure code: SCIP-CARD-2 measure name: surgery patients who were taking heart drugs called beta blockers before coming to the hospital&#54; who were kept on the beta blockers during the period just before and after their surgery score: nan sample: nan stateavg: AL_SCIP-CARD-2 
ROW: 2 provider number: 10018 hospital name: CALLAHAN EYE FOUNDATION HOSPITAL address1: 1720 UNIVERSITY BLVD address2: nan address3: nan city: BIRMINGHAM state: AL zip code: 35233 county name: JEFFERSON

Llama.generate: prefix-match hit


1. Address1 is missing in ROW 1. (value = "nan")
  2. There should be a comma after "city" in ROW 1. (value = "BIRMINGHAM, AL")
  3. The word "caxxed" in the measure name of ROW 1 is misspelled as "caxxed". (value = "beta blockers")
  4. There should be a comma after "phone number" in ROW 1. (value = "2053258100, hospital type: Acute Care Hospitals")
  5. The word "hospitax" in the county name of ROW 1 is misspelled as "hospitax". (value = "JEFFERSON, AL")
  6. There should be a comma after "condition" in ROW 1. (value = "Surgical Infection Prevention, measure code: SCIP-CARD-2")
  7. The value of the measure name in ROW 1 is misspelled as "sxury patients". (value = "surgery patients")



In [ ]:
def serialize_row(row: pd.Series) -> str:
  result = ""
  for index, value in row.items():
    result += f"{index}: {value} "
  return result

def serialize_table(table: pd.DataFrame, max_rows = 10) -> str:
  result = ""
  counter = 0
  for index, row in table.iterrows():
    counter += 1
    if counter > max_rows:
      return result
    result += serialize_row(row)
    result += "\n"

In [ ]:
# print(ask_llama(f"""Here are some correct data entries:

# {serialize_table(flights_clean, 20)}

# Find the errors in the following data:

# {serialize_table(flights_dirty, 100)}"""))


In [ ]:
from typing import Tuple, List
from random import sample

def serialize_flights_table(table: pd.DataFrame, max_rows = 10) -> Tuple[str, List[int]]:
  result = ""
  counter = 0
  index_range = sample(range(table["tupleid"].min(), table["tupleid"].max()), max_rows)
  random_tuples = table.loc[table["tupleid"].isin(index_range)]
  current_tuple = 0
  for index, row in random_tuples.iterrows():
    if current_tuple != row["tupleid"]:
      current_tuple = row["tupleid"]
      result += "\n"
    # result += f"\nIndex: {current_tuple} "
    result += f"{row['attribute']}: "
    result += f"{row['value']} "
  return (result, index_range)
serialize_flights_table(flights_clean, 2)

KeyError: ignored

In [ ]:
# print(ask_llama(f"""Here are some correct data entries:

# {serialize_flights_table(flights_clean, 5)[0]}

# Here are some erronous data entries:

# {serialize_flights_table(flights_dirty, 30)[0]}

# Please find and report the errors."""))

In [ ]:
from random import sample
import numpy as np

def generate_compare_df_flights(indexes: List[int]) -> pd.DataFrame:
  clean_tuples = flights_clean.loc[flights_clean["tupleid"].isin(indexes)]
  dirty_tuples = flights_dirty.loc[flights_dirty["tupleid"].isin(indexes)]
  compare_df = dirty_tuples.copy()
  compare_df['error'] = np.where(clean_tuples['value'] == dirty_tuples['value'], False, True)
  return compare_df

def generate_example_string_flight_data(clean: pd.DataFrame, dirty: pd.DataFrame, max_rows: int = 5) -> str:
  result = ""
  index_range = sample(range(clean["tupleid"].min(), clean["tupleid"].max()), max_rows)
  compare_df = generate_compare_df_flights(index_range)
  for index in index_range:
    current_df = compare_df.loc[compare_df['tupleid'] == index]
    row_text = ""
    error_text = ""
    for id, row in current_df.iterrows():
      row_text += f"{row['attribute']}: {row['value']} "
      if row['error']:
        error_text += f"{row['attribute']}, "
        # error_text += f"{row['attribute']}: Explanation of the error, "
    if error_text == "":
      error_text = "NO ERROR"
    else:
      error_text = "ERROR: " + error_text
    result += f"INPUT:\n{row_text}\n OUTPUT:\n{error_text}\n"
  return result

# generate_example_string_flight_data(flights_clean, flights_dirty, 1)

In [ ]:
input_row, input_indexes = serialize_flights_table(flights_dirty, 1)

print(ask_llama(f"Find the errors just as the example and explain what the errors are.\n\n{generate_example_string_flight_data(flights_clean, flights_dirty)}\n\nINPUT: {input_row}\nOUTPUT:"))
print('===============')
print(generate_compare_df_flights(input_indexes))

KeyError: ignored